In [1]:
# import pandas as pd
import numpy as np
import random
import math

Suppose the triangle has Pythagorean triple $(a, b, c)$ as its sides, with $a < b < c$. Given a limit $L$, we are looking for triangles such that $a + b + c < L$ such that $(b-a)^2 \, | \, c^2$. That is, $\exists k \in \mathbb{Z}$ such that
$$
(b-a)^2 \, | \, c^2 \iff (b-a) \, | \, c.
$$

Further, with a little manipulation, we get (using the Pythagorean Theorem) if
$$
(b-a)^2 \, | \, c^2 \implies (b-a)^2 \, | \, [c^2 - (b-a)^2] \implies (b-a)^2 \, | \, 2ab \implies b-a \, | \, 2ab.
$$

From the second one, we can see $b-a \, | \, 2ab$. Suppose $(a,b,c)$ is a primitive Pythagorean triple. Then $\gcd(a,b) = 1$. Further, we know exactly one of $a$ or $b$ is even. Let $p$ be some prime factor of $ab$. Then $p \, | \, a$ or $p \, | \, b$, but not both (since $a$ and $b$ are coprime). Thus $\gcd(b-a, ab) = 1$ since no prime factors can divide both $a$ and $b$. But also, $b-a$ is odd (since one is even and one is odd), so $\gcd(b-a, 2ab) = 1$ as well. But solutions would require that $b-a \, | \, 2ab$. If \gcd(b-a, 2ab) = 1$ and $b-a \, | \, 2ab$, then it follows that $b-a = \pm 1$.

For any multiple of primitive Pythagorean triple $(a,b,c)$ and integer $k \in \mathbb{Z}$, $(na, nb, nc)$ for some $n \in \mathbb{N}$, we have
$$
k(b-a)^2 = c^2 \iff kn^2(b - a)^2 = n^2c^2 \iff k(nb - na)^2 = (nc^2).
$$

This means we only need to check if a primitive triple satisfies the property for some integer $k$, and then we know all its multiples also satisfy, and we can count them by simply summing $\lfloor \frac{L}{a + b + c} \rfloor$. Additionally, if a primitive triple does not satisfy the property, we know that none of the multiples will satisfy it either.



In [5]:
# cached version of gcd
cache = {}
def gcd(a,b):
    if not a:
        return b

    if (a,b) not in cache:
        cache[(a,b)] = gcd(b % a, a)
    
    return cache[(a,b)]

# Euclid's formula
def euclid_pythagorean(max_perimeter):
    if max_perimeter < 12:
        return []

    triples = []
    to_check = [(2, 1)]
    seen = set()
    while to_check:
        m,n = to_check.pop(0)
        if (m,n) in seen: continue

        seen.add((m,n))

        # require gcd = 1 for primitive triples
        # require one exactly one of m or n to be even
        if gcd(m,n) == 1 and (m + n) % 2 == 1:
            a,b,c = m*m - n*n, 2*m*n, m*m + n*n
            
            if a+b+c >= max_perimeter: continue

            triples.append((min(a,b), max(a,b), c))

        if m > n + 1:
            to_check.append((m, n+1))
        to_check.append((m+1, n))

    return triples

# calculate pythagorean primitives
trips = euclid_pythagorean(10**8)

# use that b-a | c to count the primitives
count = 0
for a,b,c in trips:
    if c % (b-a) == 0:
        count += 10**8 // (a + b + c)
count

After finding the solution, I see that all the solutions have $b-a = 1$, which means there are actually no possible tilings (at least in this short pattern) where $b-a$ is actually a factor of $c$. Then looking up the hypotenuses as a series on OEIS, and I found [a few awesome series][1] that must be related to this problem.

First, one method I tried to speed up the Pythagorean generation (it actually ended up being slower than good old Euclid's formula on my machine!) was the Berggren [tree of primitive Pythagorean triples][2]. One person on OEIS noted that the hypotenuses are the ones on the "middle branch" of the Berggren tree every time! This works in particular because the middle branch keeps $b-a = 1$ every time (i.e., the middle child always preserve $b-a$ value from the parent).

Second, because $b-a = 1$, that means $a$ and $b$ are consecutive, so we have $a^2 + (a + 1)^2 = c^2$. The sum of consecutive squares is actually the definition of [centered square numbers][3]. So what this means is if $(a,b,c)$ is a solution, then $c^2$ must be a centered square number.

[1]: https://oeis.org/A001653
[2]: https://en.wikipedia.org/wiki/Tree_of_primitive_Pythagorean_triples
[3]: https://en.wikipedia.org/wiki/Centered_square_number

In [6]:
# Euclid's formula modified to only search b-a == 1 - a little over 1 min
def euclid_modified(max_perimeter):
    if max_perimeter < 12:
        return []

    triples = []
    to_check = [(2, 1)]
    seen = set()
    while to_check:
        m,n = to_check.pop(0)
        if (m,n) in seen: continue

        seen.add((m,n))

        # require gcd = 1 for primitive triples
        # require one exactly one of m or n to be even
        if gcd(m,n) == 1 and (m + n) % 2 == 1:
            a,b,c = m*m - n*n, 2*m*n, m*m + n*n
            
            # only includea solutions
            if a+b+c >= max_perimeter: continue

            if abs(b-a) == 1:
                triples.append((min(a,b), max(a,b), c))

        if m > n + 1:
            to_check.append((m, n+1))
        to_check.append((m+1, n))

    return triples

sols = euclid_modified(10**8)
count = 0
for a,b,c in sols:
    count += 10**8 // (a + b + c)
count

10057761

In [3]:
# using centered square numbers - about 45 seconds with 10 sec spent just calculating all the squares
perfect_squares = [i*i for i in range(10**8//3)]
centered_square_numbers = {perfect_squares[i] + perfect_squares[i+1]: i for i in range(len(perfect_squares)-1)}
count = 0
for c, c2 in enumerate(perfect_squares):
    if c < 5:
        continue

    try:
        ind = centered_square_numbers.index(c2)
        print(ind, ind + 1, c)
        count += 10**8 // (2*ind + 1 + c)
    except:
        pass

count

5 3
29 20
169 119
985 696
5741 4059
33461 23660
195025 137903
1136689 803760
6625109 4684659


0

In [9]:
# Berggren tree solution - instant!
count = 0
sols = [(3,4,5)]
while sum(sols[-1]) < 10**8:
    a,b,c = sols[-1]
    sols.append((a + 2*b + 2*c, 2*a + b + 2*c, 2*a + 2*b + 3*c))
    count += 10**8 // (a + b + c)
print(sols)
count

10057761